# Nétoyage des données de résultat du 1er et 2ème tour

In [ ]:
import pandas as pd
import re

base_path = '../data_brut/result'
first_tour_dataframes = []
second_tour_dataframes = []


# Liste des noms de fichiers jusqu'en 2012
file_names_old_format = [
    'cdsp_presi1965t1_circ.csv', 'cdsp_presi1965t2_circ.csv',
    'cdsp_presi1969t1_circ.csv', 'cdsp_presi1969t2_circ.csv',
    'cdsp_presi1974t1_circ.csv', 'cdsp_presi1974t2_circ.csv',
    'cdsp_presi1981t1_circ.csv', 'cdsp_presi1981t2_circ.csv',
    'cdsp_presi1988t1_circ.csv', 'cdsp_presi1988t2_circ.csv',
    'cdsp_presi1995t1_circ.csv', 'cdsp_presi1995t2_circ.csv',
    'cdsp_presi2002t1_circ.csv', 'cdsp_presi2002t2_circ.csv',
    'cdsp_presi2007t1_circ.csv', 'cdsp_presi2007t2_circ.csv',
    'cdsp_presi2012t1_circ.csv', 'cdsp_presi2012t2_circ.csv',
]

# Liste des noms de fichiers de 2017 à 2022
file_names_new_format = [
    'cdsp_presi2017t1_circ.csv', 'cdsp_presi2017t2_circ.csv',
    'cdsp_presi2022t1_circ.csv', 'cdsp_presi2022t2_circ.csv',
]

for file_name in file_names_old_format + file_names_new_format:
    # Construire le chemin complet vers le fichier CSV
    csv_file_path = f"{base_path}/{file_name}"
    
    # Lire le fichier CSV dans un DataFrame
    df = pd.read_csv(csv_file_path, encoding='UTF-8-sig', sep=',')


    # Utiliser une expression régulière pour trouver une année plausible dans le nom de fichier
    match = re.search(r'(\d{4})t\d', file_name)
    if match:
        year = int(match.group(1))  # Convertir l'année trouvée en entier
        df['Année'] = year
    else:
        year = None
    
    if 't1' in file_name:
        tour_dataframes = first_tour_dataframes
    elif 't2' in file_name:
        tour_dataframes = second_tour_dataframes
    else:
        continue


    if file_name in file_names_new_format:
        # Rename columns 
        df.rename(columns={
            'Code du département': 'Code département',
            'Code de la circonscription': 'circonscription',
            'Libellé du département': 'département',
        }, inplace=True)
        # Upper columns Libellé du département


    # Convertir 'Code département' en entier de manière sûre
    df['Code département'] = pd.to_numeric(df['Code département'], errors='coerce').fillna(0).astype(int)
    # Convertir 'circonscription' en entier de manière sûre
    df['circonscription'] = pd.to_numeric(df['circonscription'], errors='coerce').fillna(0).astype(int)
    
    if year < 1986:
        circons_to_include = [1, 2]
    else:
        circons_to_include = [1, 2, 3]

    # Filtrer sur le département 34 et sur les circonscriptions 1 et 2
    if file_name in file_names_old_format:
        df_montpellier = df[(df['Code département'] == 34) & (df['circonscription'].isin(circons_to_include))]
    else:
        df_montpellier = df[(df['Code département'] == 34) & (df['circonscription'].isin(circons_to_include))]

    # Ajouter le DataFrame filtré à la liste
    tour_dataframes.append(df_montpellier)

# Concatenate and save data for each tour
first_tour_data = pd.concat(first_tour_dataframes, ignore_index=True)
second_tour_data = pd.concat(second_tour_dataframes, ignore_index=True)

# regroup data by year
first_tour_data = first_tour_data.groupby('Année').sum().reset_index()
second_tour_data = second_tour_data.groupby('Année').sum().reset_index()


print(first_tour_data.head())  # Preview first tour data
print(second_tour_data.head())  # Preview second tour data

first_tour_data.to_csv('../dataset/first_tour_data.csv', index=False)
second_tour_data.to_csv('../dataset/second_tour_data.csv', index=False)
# to parquet
first_tour_data.to_parquet('../dataset/first_tour_data.parquet', index=False)
second_tour_data.to_parquet('../dataset/second_tour_data.parquet', index=False)




# Affectation d'une étiquette à chaque candidat

In [76]:

# Dictionnaire des candidats et leur orientation politique
candidate_to_orientation = {
    'Extrême gauche': [
        'KRIVINE (LCR)', 'LAGUILLER (LO)', 'BESANCENOT (LCR)', 'ARLETTY (LO)', 'DUCLOS (PCF)', 'BUFFET (PC)',
        'HUE (PCF)', 'LAJOINIE (PCF)', 'MARCHAIS (PCF)', 'MGBADA (AEI)', 'Nathalie ARTHAUD (LO)', 'Philippe POUTOU (NPA)',
        'Jean-Luc MELENCHON (FG)', 'Fabien ROUSSEL', 'BARBU (DIV)', 'DUCATEL (DIV)', 'MULLER (MDSR)',
        'BOUCHARDEAU (PSU)', 'SEBAG (DIV)', 'BOUSSEL (MPT)', 'JUQUIN ( )', 'WAECHTER (V)', 'BARRE (UDF)',
        'GLUCKSTEIN (PT)', 'Philippe POUTOU', 'Jacques CHEMINADE', 'Nathalie ARTHAUD', 'Philippe POUTOU', 'François ASSELINEAU','CHEMINADE (POE)','Jacques CHEMINADE (SP)','SCHIVARDI (PT)','GLUCKSTEIN (PT)','BOVE','Jean-Luc MELANCHON'
    ],
    'Gauche': [
        'MITTERRAND (CIR)', 'ROCARD (PSU)', 'MITTERRAND (PS)', 'JOSPIN (PS)', 'HOLLANDE (PS)', 'ROYAL (PS)',
        'Hamon (PS)', 'Anne HIDALGO', 'DEFFERRE (SFIO)', 'CREPEAU (MRG)', 'LALONDE (ECO)', 'TAUBIRA (PRG)',
        'CHEVENEMENT (P. REP)', 'MAMERE (VERTS)', 'VOYNET (VERTS)', 'Eva JOLY (EELV)', 'François HOLLANDE (PS)',
        'Yannick JADOT', 'Benoît HAMON', 'DUMONT (ECO)'
    ],
    'Droite': [
        'DE GAULLE (UNR)', 'POMPIDOU (UDR)', 'CHABAN-DELMAS (UDR)', 'GISCARD D\'ESTAING (RI)', 'CHIRAC (RPR)',
        'BALLADUR (UDF)', 'SARKOZY (UMP)', 'François FILLON', 'Nicolas DUPONT-AIGNAN', 'Valérie PECRESSE', 'LECANUET (MRP)',
        'POHER (CD)', 'GISCARD D\'ESTAING (UDF)', 'DEBRE (DVD)', 'GARAUD (DVD)', 'DE VILLIERS (MPF)',
        'LEPAGE (CAP 21)', 'BAYROU (UDF)', 'MADELIN (DL)', 'CHIRAC (UMP)', 'François BAYROU (MODEM)', 'Nicolas SARKOZY (UMP)',
        'Nicolas DUPONT-AIGNAN (DLR)', 'Valérie PÉCRESSE', 'Jean LASSALLE','RENOUVIN (NAR)','Emmanuel MACRON','DE VILLIERS MPF)','NIHOUS (CNPT)'
    ],  
    'Extrême droite': [
        'TIXIER-VIGNANCOUR (EXD)', 'LE PEN (FN)', 'MEGRET (MNR)', 'Marine LE PEN', 'MARCILHACY (DVD)',
        'ROYER (DVD)', 'HERAUD (DIV)', 'BOUTIN (UDF DISS.)', 'SAINT-JOSSE (CNPT)', 'Marine LE PEN (FN)', 'éric ZEMMOUR',
    ],
}



# Transformation de chaque candidat en 1 partie politique

In [ ]:
#chargement du parquet 
df = pd.read_csv('../dataset/first_tour_data.csv',encoding='utf-8-sig')
start_col_index = df.columns.get_loc('circonscription') + 1  # Début après 'circonscription'

# Vérifier et convertir les colonnes
for col in df.columns[start_col_index:]:
    try:
        print(f"Traitement de {col}, type actuel: {df[col].dtype}")
        df[col] = pd.to_numeric(df[col], errors='coerce').astype('Int64')
    except Exception as e:
        print(f"Erreur lors de la conversion de {col}: {str(e)}")

# Initialisation des colonnes de regroupement politique
df['Extrême gauche'] = 0
df['Gauche'] = 0
df['Droite'] = 0
df['Extrême droite'] = 0

# Regrouper les résultats des candidats sous les nouvelles colonnes politiques
for orientation, candidates in candidate_to_orientation.items():
    for candidate in candidates:
        if candidate in df.columns:
            df[orientation] += df[candidate]
            df.drop(columns=[candidate], inplace=True)


# Sauvegarder le nouveau DataFrame
df.to_csv("../dataset/dataframe_politique.csv", index=False,encoding='utf-8-sig')

In [ ]:
# Ajout du fichier sur le pouvoir d'achat

In [ ]:
# Charger le fichier CSV du pouvoir d'achat
pouvoir_achat = pd.read_csv('chemin/vers/fichier_pouvoir_achat.csv', delimiter=';', usecols=['Année', "Pouvoir d'achat par unité de consommation", "Pouvoir d’achat du revenu disponible brut"])

# Nettoyer les données pour éliminer les lignes contenant trop de points-virgules ou de champs vides
pouvoir_achat = pouvoir_achat.dropna(how='all')
